In [2]:
%load_ext autoreload
%autoreload 2

import warnings
import cv2
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
#import selective_search as ss
from box_ops import nms
from data import WasteSet, get_waste, standardize_inv
#from model import get_resnet
from selectivesearch import SelectiveSearch
from utils import show_annotation, show_cmap

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
train_dataset, _, _, _, _, _ = get_waste(8,
                                         num_workers=0,
                                         data_augmentation=True,
                                         supercategories=True)

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
loading annotations into memory...
Done (t=0.15s)
creating index...
index created!
loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
loading annotations into memory...
Done (t=0.16s)
creating index...
index created!


In [ ]:
def get_bbox_(target):
    """
    Gets the resized bounding box from the target(s) in the data.
    A singular target is a dict that can have multiple object bboxes inside.
    """
    out = []
    for t in range(len(target)):
        im_h, im_w = target[t]['size']
        #print(f"Target {t} has size {target[t]['size']}")
        #print("Resizing the bbox...")
        bbox = [x, y, w, h] = target[t]['bbox']
        resized_bbox = [x * im_w, y * im_h, w * im_w, h * im_h]
        out.append(resized_bbox)
    return out

def get_iou(box1, box2):
    # Extract coordinates from bounding boxes
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    # Calculate the coordinates of the intersection rectangle
    intersection_x1 = max(x1, x2)
    intersection_y1 = max(y1, y2)
    intersection_x2 = min(x1 + w1, x2 + w2)
    intersection_y2 = min(y1 + h1, y2 + h2)
    
    # Calculate the area of the intersection rectangle
    intersection_area = max(0, intersection_x2 - intersection_x1) * max(0, intersection_y2 - intersection_y1)
    
    # Calculate the area of each bounding box
    area_box1 = w1 * h1
    area_box2 = w2 * h2
    
    # Calculate the union area
    union_area = area_box1 + area_box2 - intersection_area
    
    # Calculate the IoU
    iou = intersection_area / union_area    
    return iou

In [4]:
# Change number of region proposals we return
# Plot it versus the recall of each region given a threshold value ie. 0.5?
# We calculate the recall using IoU?

# We need to somehow get recall into the mix, look at plots like
# https://www.google.com/url?sa=i&url=https%3A%2F%2Fwww.researchgate.net%2Ffigure%2FRecall-versus-Intersection-over-Union-IoU-threshold-on-the-Munich-Vehicle-test-set-a_fig6_313593125&psig=AOvVaw3SHomc2QxKoXEwjC1jN-4Y&ust=1687253148064000&source=images&cd=vfe&ved=0CBEQjRxqFwoTCKCT7vaBz_8CFQAAAAAdAAAAABAQ

from tqdm import tqdm
threshold = 0.5

rcs = []
kk = 0
tars = []
glob_len_tar = 0
for im, tar in tqdm(train_dataset):
    print('hey')


    ss = SelectiveSearch(mode='f', nkeep=5000)
    im, _ = standardize_inv(im, None)
    boxes = ss((np.moveaxis(im.numpy(), 0, 2) * 255).astype(np.uint8))


    n_rps = np.flip(np.arange(0, 5000, 100)).tolist()
    #print(f"Number of boxes proposed: {len(boxes)}")
    #print(f"Global minimum comparison: {glob_min}")
    targets = get_bbox_(tar)
    len_tar = len(targets)
    tars.append(len_tar)
    glob_len_tar += len_tar
    print(f"# targets: {len_tar}")

    for n in n_rps:
        print(f"Here is n: {n}")
        print(f"Here is len(boxes) {len(boxes)}")
        if len(boxes) < n:
            print("if")
            recall = 1.0
            rcs.append(recall)
            print(recall)
        else:
            print("else")
            ious = [[] for x in range(0, len(targets))]
            catcher = []
            #print(f"Now n is: {n}")
            boxes_ = boxes[:n]

            for i in range(0,len(boxes_)):
                # Get iou between the target
                for k, t in enumerate(targets):
                    iou = get_iou(boxes[i], t)
                    ious[k].append(iou)
            try:
                catcher.extend([max(map(float, i)) for i in ious])
            except:
                catcher.extend([0.0, 0.0]) # edge case
            #print(f"Catcher after first pass: {catcher}")
            iou_above = [val for val in catcher if val >= threshold]
            #print(f"iou_above after first pass: {iou_above}")

        ### NOTE TO SELF: NOW DO IT FOR ALL IMAGES IN THE TRAINING SET
        ### HOW DO WE HANDLE IOU FROM DIFF IMAGES ETC? AVERAGE? 
        ### WE AT LEAST NEED #GT TO BE SUM OF ALL THE TARGETS WE FIND
        ### SO ... WHAT DOES THIS MEAN FOR THE MAX(IoU(target(t)))???

        # Maximally IoU above threshold for GT from RP
        # -------------------------------------------
        # Total number of ground truths in the image

            recall = len(iou_above)/len_tar
            print(recall)
            #print(f"Recall: {recall}")
            #print( len(iou_above), recall, catcher )
            rcs.append(recall)
            #print(f"Recalls: {rcs}")
            #print(iou_above)

    kk += 1
    print(f"kk = {kk}")
    if kk == 100:
        break



  0%|                                                                                          | 0/1050 [00:00<?, ?it/s]

hey


  0%|                                                                                  | 1/1050 [00:02<40:34,  2.32s/it]

# targets: 5
Here is n: 4900
Here is len(boxes) 324
if
1.0
Here is n: 4800
Here is len(boxes) 324
if
1.0
Here is n: 4700
Here is len(boxes) 324
if
1.0
Here is n: 4600
Here is len(boxes) 324
if
1.0
Here is n: 4500
Here is len(boxes) 324
if
1.0
Here is n: 4400
Here is len(boxes) 324
if
1.0
Here is n: 4300
Here is len(boxes) 324
if
1.0
Here is n: 4200
Here is len(boxes) 324
if
1.0
Here is n: 4100
Here is len(boxes) 324
if
1.0
Here is n: 4000
Here is len(boxes) 324
if
1.0
Here is n: 3900
Here is len(boxes) 324
if
1.0
Here is n: 3800
Here is len(boxes) 324
if
1.0
Here is n: 3700
Here is len(boxes) 324
if
1.0
Here is n: 3600
Here is len(boxes) 324
if
1.0
Here is n: 3500
Here is len(boxes) 324
if
1.0
Here is n: 3400
Here is len(boxes) 324
if
1.0
Here is n: 3300
Here is len(boxes) 324
if
1.0
Here is n: 3200
Here is len(boxes) 324
if
1.0
Here is n: 3100
Here is len(boxes) 324
if
1.0
Here is n: 3000
Here is len(boxes) 324
if
1.0
Here is n: 2900
Here is len(boxes) 324
if
1.0
Here is n: 2800
Here 

  0%|▏                                                                                 | 2/1050 [00:05<53:04,  3.04s/it]

0.3333333333333333
Here is n: 800
Here is len(boxes) 2653
else
0.3333333333333333
Here is n: 700
Here is len(boxes) 2653
else
0.3333333333333333
Here is n: 600
Here is len(boxes) 2653
else
0.3333333333333333
Here is n: 500
Here is len(boxes) 2653
else
0.3333333333333333
Here is n: 400
Here is len(boxes) 2653
else
0.3333333333333333
Here is n: 300
Here is len(boxes) 2653
else
0.0
Here is n: 200
Here is len(boxes) 2653
else
0.0
Here is n: 100
Here is len(boxes) 2653
else
0.0
Here is n: 0
Here is len(boxes) 2653
else
0.0
kk = 2
hey


  0%|▏                                                                               | 2/1050 [00:08<1:12:54,  4.17s/it]


KeyboardInterrupt: 

In [ ]:
len(rcs)

In [ ]:
recalls = np.array(rcs).reshape(kk, len(rcs) // kk)
n_tar = np.array(tars).reshape(-1, 1)
wmean_recalls = (recalls * n_tar).sum(0) / glob_len_tar

In [ ]:
plt.plot(np.flip(n_rps).tolist(), wmean_recalls[::-1], color='magenta', mfc='pink' ) #plot the data
#plt.xticks(range(0,50+1, 100)) #set the tick frequency on x-axis

plt.xlabel('# Region Proposals')
plt.ylabel('Recall')
plt.title('IoU @ 0.5')
plt.show() #display the graph